In [1]:
#%matplotlib notebook
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
import networkx as nx
from tqdm import tqdm
from sklearn.model_selection import train_test_split

%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


In [2]:
data_path="/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/"

In [3]:
df_path=glob.glob(data_path+"***")

In [4]:
df_path

['/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_A',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_B',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_C',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_D',
 '/home/nvekariy/signal_code/datasets/Bonn_seizure_dataset/Set_E']

In [5]:
LABEL_C1 = 0
LABEL_C2 = 1

In [6]:
def load_data():
    data = []
    nbFiles = 0
    df=glob.glob(df_path[1]+"/***")
    for fname in tqdm(df):
        img = np.loadtxt(fname)
        data.append([img, np.array(LABEL_C1)])
        nbFiles+=1
        
    df1=glob.glob(df_path[-1]+"/***")
    for fname in tqdm(df1):
        img = np.loadtxt(fname)
        data.append([np.array(img), np.array(LABEL_C2)])
        nbFiles+=1
    
    return data

In [7]:
data = load_data()
print(len(data), "Files")

100%|██████████| 100/100 [00:02<00:00, 46.01it/s]

200 Files


In [8]:
from sklearn.utils import shuffle
data = shuffle(data,random_state=40)


X = np.array([d[0] for d in data])
Y = np.array([d[1] for d in data])



In [9]:
X.shape

(200, 4097)

In [10]:
def min_max_normalize(matrix):
    """
    Perform min-max normalization on a 2D array (matrix).
    """
    min_vals = np.min(matrix, axis=0)
    max_vals = np.max(matrix, axis=0)
    normalized = (matrix - min_vals) / (max_vals - min_vals)
    return normalized

In [11]:
X_n=np.array([min_max_normalize(i) for i in X])


In [12]:
X_temp=[]
Y_temp=[]
for i in range(len(X)):
        raw=X_n[i]
        X_temp.append(raw[:1024])
        X_temp.append(raw[1024:2048])
        X_temp.append(raw[2048:3072])
        X_temp.append(raw[3072:4096])
        
        Y_temp.extend(np.repeat(Y[i],4))

        

In [13]:
X_temp=np.array(X_temp)
Y_temp=np.array(Y_temp)

In [14]:
print(X_temp.shape)
print(Y_temp.shape)

(800, 1024)
(800,)


In [15]:
from sklearn.utils import shuffle
X_s,Y_s = shuffle(X_temp,Y_temp,random_state=40)


In [16]:
unique_elements, element_counts = np.unique(Y_s, return_counts=True)

print("Number of unique elements:", len(unique_elements))
print("Unique elements:", unique_elements)
print("Element counts:", element_counts)


Number of unique elements: 2
Unique elements: [0 1]
Element counts: [400 400]


In [17]:
def calculate_visibility(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "dual_natural", weight='tan')
        vs_out=np.array(vs_out)
        temp1.append(vs_out)
        
    return temp1

In [18]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [19]:
df=np.array_split(X_s, num_cores)


In [20]:
print(len(df[0]))

17


In [21]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_visibility, df)

pool.close()
pool.join()

In [22]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [23]:
print(len(x_adj))

800


In [24]:
x_adj=np.array(x_adj)

In [25]:
def calculate_g(data):
    temp2=[]
    for vs_out in data:
        a = np.sum(vs_out, axis=0).reshape(-1, 1)
        b = np.sum(vs_out != 0, axis=1).reshape(-1, 1)
        temp2.append(np.concatenate((a, b), axis=1))
        
    return temp2

In [26]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
17


In [27]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(calculate_g, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 11.220099210739136 seconds ---


In [28]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [29]:
x_g=np.array(x_g)


In [30]:
print(x_g.shape)

(800, 1024, 2)


In [31]:
x_g_flat=np.array([np.array(i.T.flatten()).ravel() for i in x_g])


In [32]:
x_train, x_test, y_train, y_test = train_test_split(x_g_flat, Y_s, test_size=0.30, random_state=42)


In [33]:
print(x_train.shape)
print(x_test.shape)

(560, 2048)
(240, 2048)


In [34]:
import tensorflow as tf


2023-08-12 09:05:28.813491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 09:05:28.944243: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-12 09:05:28.948757: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-

In [35]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-12 09:05:50.441665: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-12 09:05:50.441695: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-12 09:05:50.441715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0155): /proc/driver/nvidia/version does not exist
2023-08-12 09:05:50.442477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow

In [36]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2048, 32)          128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1024, 32)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1024, 32)          3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 512, 32)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 512, 32)           3104      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 256, 32)          0

In [37]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [38]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F8.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [39]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
18/18 [==============================] - 2s 30ms/step - loss: 0.8718 - acc: 0.4964 - val_loss: 0.6712 - val_acc: 0.5875
Epoch 2/30
18/18 [==============================] - 0s 14ms/step - loss: 0.6671 - acc: 0.5714 - val_loss: 0.6168 - val_acc: 0.7625
Epoch 3/30
18/18 [==============================] - 0s 14ms/step - loss: 0.5440 - acc: 0.7000 - val_loss: 0.4087 - val_acc: 0.8542
Epoch 4/30
18/18 [==============================] - 0s 14ms/step - loss: 0.3667 - acc: 0.8482 - val_loss: 0.2911 - val_acc: 0.8917
Epoch 5/30
18/18 [==============================] - 0s 14ms/step - loss: 0.3017 - acc: 0.8607 - val_loss: 0.4181 - val_acc: 0.8042
Epoch 6/30
18/18 [==============================] - 0s 14ms/step - loss: 0.3530 - acc: 0.8375 - val_loss: 0.3939 - val_acc: 0.8042
Epoch 7/30
18/18 [==============================] - 0s 14ms/step - loss: 0.2328 - acc: 0.9071 - val_loss: 0.2834 - val_acc: 0.8750
Epoch 8/30
18/18 [==============================] - 0s 14ms/step - loss: 0.2547 - a

Epoch 29/30
18/18 [==============================] - 0s 14ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.1960 - val_acc: 0.9625
Epoch 30/30
18/18 [==============================] - 0s 14ms/step - loss: 6.6132e-04 - acc: 1.0000 - val_loss: 0.1636 - val_acc: 0.9708


In [40]:
model3.save_weights('/home/nvekariy/signal_code/New_bone_eeg_code_B_E_h/F8.h5')


In [41]:
model3.load_weights('/home/nvekariy/signal_code/New_bone_eeg_code_B_E_h/F8.h5')


In [42]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

8/8 [==============================] - 0s 4ms/step


In [43]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[110   1]
 [  6 123]]


In [44]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [45]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

97.08333333333333
97.0747505702495
97.08333333333333
97.17429087875416
